### Homework2


Here we define a few parameters for our little experiment

## Homework2

## Homework 

In [1]:
import torch

# we will use with Distil-BERT
language_model_name = "distilbert-base-cased"
# this GPU should be enough for this task to handle 32 samples per batch
batch_size = 32
# we keep num_workers = min(4 * number of GPUs, number of cores)
# tells the data loader how many sub-processes to use for data loading
num_workers = 2
# optim
learning_rate = 2e-4
weight_decay = 0.0
transformer_learning_rate = 1e-5
transformer_weight_decay = 0.0
# training
epochs = 3
device = "cuda" if torch.cuda.is_available() else "cpu"

/home/francesco/anaconda3/envs/nlp2022-hw2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dataset

In [4]:
import os
import json
import logging
import torch
from torch.utils.data import DataLoader,Dataset
import random
from typing import Dict

class DatasetCustom(Dataset):
 
 
    def __init__(self,language) -> None:
        self.path_root = 'data'
        self.load_data(language)
        self.args_roles,self.list_broken_id = self.list_arg_roles()
        print(self.args_roles)
        self.pos_list,_ = self.list_pos()
        
        pass
        
    def load_data(self, language : str):
        path_train = os.path.join(self.path_root,str(language),'train.json')
        path_dev = os.path.join(self.path_root,str(language),'dev.json')

        data_file_train = open(path_train)
        data_file_dev = open(path_dev)
  

        data_train = json.load(data_file_train)
        data_dev = json.load(data_file_dev)

        list_train = []
        list_dev = []

        for data in data_train:
            list_train.append(data_train[data])
        

        for data in data_dev:
            list_dev.append(data_dev[data])

        

        self.data_train = list_train
        self.data_dev = list_dev
    
    def __len__(self):
        return len(self.data_train)

    def __getitem__(self, id : int):

        flag = False
        if id in self.list_broken_id :
            flag = True
            while flag == True:

                rand_id = random.randint(0, len(self.data_train))
                
                if rand_id in self.list_broken_id :
                    pass
                else :
                    flag = False
                    id = rand_id        


        data = self.pre_processing(self.data_train[id])
        data = self.processig(data)
        return data
        
    def pre_processing(self, data:dict):
        data_list = []
        for role in data["roles"]:
            dictionary = dict()
            dictionary["words"] = data["words"]
            dictionary["role"] = data["roles"][role]
            dictionary["pre_idx"] = role
            
            dictionary["pos_tags"] = data["pos_tags"]
            data_list.append(dictionary)    
        return data_list
    
    def processig(self,data_list:list):
        
        for dictionary in data_list:

            #dictionary["words"] = data["words"]
            dictionary["gt_arg_identification"] = self.arg_id(dictionary["role"])
            dictionary["gt_arg_classification"] = self.arg_class(dictionary["role"])
            dictionary["pos_idx"] = self.pos_idx(dictionary["pos_tags"])
        
        return data_list
   
    def list_arg_roles(self):
        list_roles = []
        list_broken_id = []
        for i,element in enumerate(self.data_train):
            flag = True
            try : roles = element["roles"]
            except : flag = False
            if flag :
                for e in roles:
                    sentence = element["roles"][e]

                    for word in sentence:
                        
                        list_roles.append(word)
                list_roles = list(set(list_roles))
            else : 
                list_broken_id.append(i)
        return list_roles,list_broken_id

    def list_pos(self):
        list_pos = []
        list_broken_id = []
        for i,element in enumerate(self.data_train):
            flag = True
            try : pos = element["pos_tags"]
            except : flag = False
            if flag :
                for e in pos:
                    list_pos.append(e)
                list_pos = list(set(list_pos))
            else : 
                list_broken_id.append(i)
        return list_pos,list_broken_id
  
    def arg_class(self,role:list):
        list_idxs = []
        for element in role:
            list_idxs.append(self.args_roles.index(element))
        

        return torch.tensor(list_idxs, dtype=torch.int64)

    def arg_id(self,role:dict):
        list_idxs = []
        for element in role:
            if element == "_":
                list_idxs.append(0)
            else :
                list_idxs.append(1)

        

        return torch.tensor(list_idxs, dtype=torch.int64)

    def pos_idx(self,pos_tags:dict):
        list_idxs = []
        for element in pos_tags:
            list_idxs.append(self.pos_list.index(element))
    
        return torch.tensor(list_idxs, dtype=torch.int64)
    


In [11]:
slr_dataset = DatasetCustom("EN")


# here we define a vocab dict to map the labels to int (and vice versa)
label_list = slr_dataset.args_roles
label_to_id = {n: i for i, n in enumerate(label_list)}
id_to_label = {i: n for n, i in label_to_id.items()}

print(label_to_id)
print(id_to_label)

['co-patient', 'goal', 'destination', 'topic', 'time', 'recipient', 'agent', 'stimulus', 'location', 'material', 'product', 'cause', 'beneficiary', 'experiencer', 'extent', 'instrument', 'theme', '_', 'co-agent', 'value', 'result', 'source', 'asset', 'attribute', 'co-theme', 'purpose', 'patient']
{'co-patient': 0, 'goal': 1, 'destination': 2, 'topic': 3, 'time': 4, 'recipient': 5, 'agent': 6, 'stimulus': 7, 'location': 8, 'material': 9, 'product': 10, 'cause': 11, 'beneficiary': 12, 'experiencer': 13, 'extent': 14, 'instrument': 15, 'theme': 16, '_': 17, 'co-agent': 18, 'value': 19, 'result': 20, 'source': 21, 'asset': 22, 'attribute': 23, 'co-theme': 24, 'purpose': 25, 'patient': 26}
{0: 'co-patient', 1: 'goal', 2: 'destination', 3: 'topic', 4: 'time', 5: 'recipient', 6: 'agent', 7: 'stimulus', 8: 'location', 9: 'material', 10: 'product', 11: 'cause', 12: 'beneficiary', 13: 'experiencer', 14: 'extent', 15: 'instrument', 16: 'theme', 17: '_', 18: 'co-agent', 19: 'value', 20: 'result', 

In [18]:
from typing import Dict
import torch
from transformers import AutoModel,AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
# load our dataset
batch_sentence = [] 
batch_preticate = [] 


# here we define our collate function
def collate_fn(batch) -> Dict[str, torch.Tensor]:
    for period in batch:
        for sentence in period :
            predicate = sentence["words"][int(sentence["pre_idx"])]
            text = " ".join(sentence["words"])
            tokens: list[str] = text.split()
            predicate: list[str] = predicate.split()
            batch_sentence.append(tokens)
            batch_preticate.append(predicate)

    batch_output = tokenizer(text = batch_sentence,text_pair = batch_preticate, padding=True, is_split_into_words=True, return_offsets_mapping=True, return_tensors="pt")
    

    barch_gt =  []
    labels = []
    for period in batch:
        for sentence in period :
            barch_gt.append(sentence["gt_arg_classification"])


    for i, label in enumerate(barch_gt):
      # obtains the word_ids of the i-th sentence
      word_ids = batch_output.word_ids(batch_index=i)
      previous_word_idx = None
      label_ids = []
      for word_idx in word_ids:
        # Special tokens have a word id that is None. We set the label to -100 so they are automatically
        # ignored in the loss function.
        if word_idx is None:
          label_ids.append(-100)
        # We set the label for the first token of each word.
        elif word_idx != previous_word_idx:
          label_ids.append(label[word_idx])
        # For the other tokens in a word, we set the label to -100 so they are automatically
        # ignored in the loss function.
        else:
          label_ids.append(-100)
        previous_word_idx = word_idx
      labels.append(label_ids)
    
    # pad the labels with -100
    batch_max_length = len(max(labels, key=len))
    labels = [l + ([-100] * abs(batch_max_length - len(l))) for l in labels]
    print(labels)

    
    return batch_output


dataloader = DataLoader(slr_dataset, batch_size=2, shuffle=False, sampler=None,
           batch_sampler=None, num_workers=0, collate_fn=collate_fn,
           pin_memory=False, drop_last=False, timeout=0,
           worker_init_fn=None, prefetch_factor=2,
           persistent_workers=False)


for i_batch, sample_batched in enumerate(dataloader):
    print("sample_batched",sample_batched)

[[-100, tensor(6), tensor(17), tensor(17), tensor(16), tensor(17), tensor(17), tensor(17), tensor(17), tensor(17), tensor(17), tensor(17), tensor(17), tensor(17), tensor(17), -100, tensor(6), -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100], [-100, tensor(17), tensor(17), tensor(17), tensor(17), tensor(17), tensor(17), tensor(12), tensor(17), tensor(17), tensor(16), tensor(17), tensor(17), tensor(17), tensor(17), -100, tensor(17), -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -1

IndexError: index 30 is out of bounds for dimension 0 with size 30

We start by building our data "section". We will do three things

- Instantiate the tokenizer
- Define our `collate_fn` that will handle the model preparation step
- Instantiate the train and validation DataLoaders

In [ ]:
from typing import Dict
import torch
from datasets import load_dataset
from transformers import AutoModel,AutoTokenizer

# load our dataset


# here we define our collate function
def collate_fn(batch) -> Dict[str, torch.Tensor]:

    batch_out = tokenizer(
        [sentence["words"] for sentence in batch],
        return_tensors="pt",
        padding=True,
        # We use this argument because the texts in our dataset are lists of words.
        is_split_into_words=True,
    )
    labels = []
    ner_tags = [sentence["ner_tags"] for sentence in batch]
    for i, label in enumerate(ner_tags):
      # obtains the word_ids of the i-th sentence
      word_ids = batch_out.word_ids(batch_index=i)
      previous_word_idx = None
      label_ids = []
      for word_idx in word_ids:
        # Special tokens have a word id that is None. We set the label to -100 so they are automatically
        # ignored in the loss function.
        if word_idx is None:
          label_ids.append(-100)
        # We set the label for the first token of each word.
        elif word_idx != previous_word_idx:
          label_ids.append(label[word_idx])
        # For the other tokens in a word, we set the label to -100 so they are automatically
        # ignored in the loss function.
        else:
          label_ids.append(-100)
        previous_word_idx = word_idx
      labels.append(label_ids)
    
    # pad the labels with -100
    batch_max_length = len(max(labels, key=len))
    labels = [l + ([-100] * abs(batch_max_length - len(l))) for l in labels]
    batch_out["labels"] = torch.as_tensor(labels)
    return batch_out

Generating train split:   0%|          | 0/14042 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3251 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3454 [00:00<?, ? examples/s]

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/63f4ebd1bcb7148b1644497336fd74643d4ce70123334431a3c053b7ee4e96ee. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

**IMPORTANT:** When using Transformers, you are dealing with sub-words inside your model, while your labels are, in token classification tasks, relative to whole words instead. You should take care of this misalignment. Things you can do:

- Rebuild the word representation from the subwords somewhere inside your model, using clever matrix operations
- Use the prediction of the first subword to represent the whole word (what we did here)

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    ner_dataset["train"],
    batch_size=batch_size,
    collate_fn=collate_fn,
    num_workers=num_workers,
    shuffle=True,
)

val_dataloader = DataLoader(
    ner_dataset["validation"],
    batch_size=batch_size,
    collate_fn=collate_fn,
    num_workers=num_workers,
    shuffle=False,
)

We now have to define our model. There are two ways:

- Wrap the transformer model inside a classic `torch.nn.Module`
- Use the various implementation offered from 🤗Transformers library

The former is very similar to what you have probably already done - think of it as a replacement for a word embedding layer. The latter instead takes advantage of the classes from 🤗Transformers library, like for instance `BertForTokenClassification` that already includes a classification layer on top of the transformer model.

For this notebook we will stick to the first approach (while you can explore the second one in the official documentation).

So, let's define our model. It needs three parameters:

- `language_model_name`: the language model to load
- `num_labels`: number of labels for the classification layer
- `fine_tune_lm`: if set to `False`, the transformer model weights are not updated during training. The advantage is that it's easier to train (less time and less memory), but performance usually drops.

In [ ]:
import torch
import torch.nn.functional as F

class NERModule(torch.nn.Module):
    def __init__(self, language_model_name: str, num_labels: int, fine_tune_lm: bool = True, *args, **kwargs) -> None:
        super().__init__()
        self.num_labels = num_labels
        # layers
        self.transformer_model = AutoModel.from_pretrained(language_model_name, output_hidden_states=True)
        if not fine_tune_lm:
            for param in self.transformer_model.parameters():
                param.requires_grad = False
        self.dropout = torch.nn.Dropout(0.2)
        self.classifier = torch.nn.Linear(
            self.transformer_model.config.hidden_size, num_labels, bias=False
        )

    def forward(
        self,
        input_ids: torch.Tensor = None,
        attention_mask: torch.Tensor = None,
        token_type_ids: torch.Tensor = None,
        labels: torch.Tensor = None,
        compute_predictions: bool = False,
        compute_loss: bool = True,
        *args,
        **kwargs,
    ) -> torch.Tensor:
        # group model inputs and pass to the model
        model_kwargs = {
          "input_ids": input_ids, 
          "attention_mask": attention_mask
        }
        # not every model supports token_type_ids
        if token_type_ids:
          model_kwargs["token_type_ids"] = token_type_ids
        transformers_outputs = self.transformer_model(**model_kwargs)
        # we would like to use the sum of the last four hidden layers
        transformers_outputs_sum = torch.stack(transformers_outputs.hidden_states[-4:], dim=0).sum(dim=0)
        transformers_outputs_sum = self.dropout(transformers_outputs_sum)
        
        logits = self.classifier(transformers_outputs_sum)

        output = {"logits": logits}

        if compute_predictions:
            predictions = logits.argmax(dim=-1)
            output["predictions"] = predictions

        if compute_loss and labels is not None:
            output["loss"] = self.compute_loss(logits, labels)

        return output

    def compute_loss(
        self, logits: torch.Tensor, labels: torch.Tensor
    ) -> torch.Tensor:
        """
        Compute the loss of the model.
        Args:
            logits (`torch.Tensor`):
                The logits of the model.
            labels (`torch.Tensor`):
                The labels of the model.
        Returns:
            obj:`torch.Tensor`: The loss of the model.
        """
        return F.cross_entropy(
            logits.view(-1, self.num_labels),
            labels.view(-1),
            ignore_index=-100,
        )

Now we can train it:

In [ ]:
from torch.optim import Adam

# instantiate the model
model = NERModule(language_model_name, len(label_to_id.keys()))
model.to(device)

# optimizer
groups = [
  {
    "params": model.classifier.parameters(),
    "lr": learning_rate,
    "weight_decay": weight_decay,
  },
  {
    "params": model.transformer_model.parameters(),
    "lr": transformer_learning_rate,
    "weight_decay": transformer_weight_decay,
  },
]
optimizer = Adam(groups)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from tqdm import tqdm

number_of_training_steps = len([b for b in train_dataloader])
for epoch in range(epochs):  # loop over the dataset multiple times
  batch_pbar = tqdm(enumerate(train_dataloader), total=number_of_training_steps)
  for i, batch in batch_pbar:
    # zero the parameter gradients
    optimizer.zero_grad()
    # forward + backward + optimize
    # move batch data to the specified device
    batch = {k: v.to(device) for k, v in batch.items()}
    # ** operator converts batch items in named arguments, (e.g. 'input_ids', 'attention_mask_ids' ...), taken as input by the model forward pass
    outputs = model(**batch)
    loss = outputs["loss"]
    loss.backward()
    optimizer.step()
    batch_pbar.set_postfix({'loss': loss.item()})

100%|██████████| 439/439 [01:06<00:00,  6.56it/s, loss=0.0662]


Let's now compute the score on the validation set:

In [ ]:
!pip install seqeval

     |████████████████████████████████| 43 kB 2.4 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=1900162fab101ff360ab471f81a58140b93616ccaaaa7c56b7ba6da430571f38
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


**seqeval** is a Python framework for sequence labeling evaluation. seqeval can evaluate the performance of chunking tasks such as named-entity recognition, part-of-speech tagging, semantic role labeling and so on. 

In [ ]:
from datasets import load_metric
# load_metric is a huggingface utility to load the most-used metrics in ML.
metric = load_metric("seqeval")

# predict
predictions, labels = [], []
for batch in tqdm(val_dataloader):
    batch = {k: v.to(device) for k, v in batch.items()}
    outputs = model(**batch, compute_predictions=True)
    y_true = batch["labels"].tolist()
    y_pred = outputs["predictions"].tolist()
    true_predictions = [
        [label_list[p] for (p, l) in zip(pred, gold_label) if l != -100]
        for pred, gold_label in zip(y_pred, y_true)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(pred, gold_label) if l != -100]
        for pred, gold_label in zip(y_pred, y_true)
    ]
    predictions += true_predictions
    labels += true_labels

# overall score print
scores = metric.compute(predictions=predictions, references=labels)
for k, v in scores.items():
    print(f"{k}: {v}")

100%|██████████| 102/102 [00:05<00:00, 17.99it/s]


LOC: {'precision': 0.9328678839957035, 'recall': 0.9455634186173109, 'f1': 0.9391727493917276, 'number': 1837}
MISC: {'precision': 0.7734294541709578, 'recall': 0.8145336225596529, 'f1': 0.7934495509772848, 'number': 922}
ORG: {'precision': 0.8393602225312935, 'recall': 0.9000745712155108, 'f1': 0.8686577905721482, 'number': 1341}
PER: {'precision': 0.971830985915493, 'recall': 0.9739413680781759, 'f1': 0.972885032537961, 'number': 1842}
overall_precision: 0.8973352950792872
overall_recall: 0.92376304274655
overall_f1: 0.9103574094037649
overall_accuracy: 0.9840543592539231


With just few epochs - and no particular parameter tuning - we have a pretty good score :)

This concludes the notebook. If you have any doubts, you can check the documentation or reach us out.